Let's start by loading the necessary libraries, and setting some configuration options for reproducibility and visualization.

In [1]:
# %%
#Import top level libraries, including the deepvelo package
import numpy as np
import scvelo as scv
import torch

from deepvelo.utils import velocity, update_dict
from deepvelo.utils.preprocess import autoset_coeff_s
from deepvelo.utils.plot import statplot, compare_plot
from deepvelo import train, Constants

# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

# set options for for visualization and verbosity
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.set_figure_params(
    "scvelo", transparent=False
)  # for beautified visualization

%load_ext autoreload
%autoreload 2


/h/hmaan/miniconda3/envs/dvelo_py_38/lib/python3.8/site-packages/deepvelo/utils/plot.py:5: DeprecationWarning: deepvelo.utils.plot is deprecated. Please use deepvelo.plot.plot instead.
  warnings.warn(


We're going to be using the Dentate Gyrus neurogenesis data from [La Manno et al. (2018)](https://doi.org/10.1038/s41586-018-0414-6) in this example. Start by loading and preprocessing the data.

In [2]:
adata = scv.datasets.dentategyrus_lamanno()
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_neighbors=30, n_pcs=30)


Filtered out 18710 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 2000 highly variable genes.
Logarithmized X.
computing neighbors
    finished (0:00:26) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:03) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


Now we're going to configure the DeepVelo model and name the experiment - we'll just call it DeepVelo for now. We're also going to empirically set the spliced correlation objective based on the data - this is recommended for best performance.

In [3]:
configs = {
    "name": "DeepVelo", # name of the experiment
    "loss": {"args": {"coeff_s": autoset_coeff_s(adata)}} # Automatic setting of the spliced correlation objective
}
configs = update_dict(Constants.default_configs, configs)


The ratio of spliced reads is 75.4% (between 70% and 85%). Suggest using coeff_s 0.75.


Now we can call the velocity and train methods to fit the model to the data. 

In [ ]:
# initial velocity
velocity(adata, mask_zero=False)
trainer = train(adata, configs)

computing velocities
    finished (0:00:09) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
Config Warning: Set to use GPU, but GPU version of DGL is not installed. Reset to use CPU instead.
building graph


INFO:train:Beginning training of DeepVelo ...


velo data shape: torch.Size([18213, 2000])


INFO:trainer:    epoch          : 1
INFO:trainer:    time:          : 21.26166820526123
INFO:trainer:    loss           : 80684.3046875
INFO:trainer:    mse            : 0.942495584487915
INFO:trainer:    epoch          : 2
INFO:trainer:    time:          : 21.131349325180054
INFO:trainer:    loss           : 16099.67578125
INFO:trainer:    mse            : 0.6484971046447754
INFO:trainer:    epoch          : 3
INFO:trainer:    time:          : 21.7674458026886
INFO:trainer:    loss           : 9191.4521484375
INFO:trainer:    mse            : 0.6198839545249939
INFO:trainer:    epoch          : 4
INFO:trainer:    time:          : 22.867656230926514
INFO:trainer:    loss           : 6672.14892578125
INFO:trainer:    mse            : 0.6180376410484314
INFO:trainer:    epoch          : 5
INFO:trainer:    time:          : 23.165233373641968
INFO:trainer:    loss           : 5399.71337890625
INFO:trainer:    mse            : 0.6397435665130615
INFO:trainer:    epoch          : 6
INFO:trai

Now that the velocity calculation is complete, we can visualize the results. We'll start by visualizing the velocity field in the embedding space.

In [ ]:
# velocity plot
scv.tl.velocity_graph(adata, n_jobs=8)
scv.pl.velocity_embedding_stream(
    adata,
    basis="tsne",
    color="clusters",
    legend_fontsize=9,
    dpi=150,  # increase dpi for higher resolution
)


We can further visualize the pseudotime estimated based on the velocity field, and plot this in the embedding space.

In [ ]:
scv.tl.velocity_pseudotime(adata)
scv.pl.scatter(
    adata,
    color="velocity_pseudotime",
    cmap="gnuplot",
    dpi=150,
)

There are a number of other visualizations and analyses that can be performed - please see the rest of the examples from the paper.
